## 고속도로

In [116]:
import pandas as pd
import numpy as np

from scipy.stats import ttest_ind
from sklearn.preprocessing import MinMaxScaler

from sklearn.cluster import KMeans

from statsmodels.formula.api import ols

In [2]:
df_1 = pd.read_csv("highway_01_07.csv")
df_2 = pd.read_csv("highway_08_12.csv")

In [3]:
df_1.head(2)

,date,StartPoint,Gyeonggi,Chungcheong,Jeolla,Gyeongsang,Gangwon
0,20140101,Chungcheong,83318.0,104617.0,4147.0,4126.0,6021.0
1,20140101,Gangwon,57398.0,5783.0,427.0,991.0,102850.0


In [4]:
df_2.head(2)

,date,StartPoint,Gyeonggi,Chungcheong,Jeolla,Gyeongsang,Gangwon
0,20140801,강원,63365.0,7044.0,892.0,1756.0,140258.0
1,20140801,경기,553781.0,110895.0,9961.0,12596.0,89665.0


In [7]:
names = df_1["StartPoint"].unique()
print(names)
print(set(df_2["StartPoint"]))

['Chungcheong' 'Gangwon' 'Gyeonggi' 'Gyeongsang' 'Jeolla']
{'강원', '전라', '경상', '충청', '경기'}


In [9]:
df_1["StartPoint"] = df_1["StartPoint"].str.replace(names[0], "충청")
df_1["StartPoint"] = df_1["StartPoint"].str.replace(names[1], "강원")
df_1["StartPoint"] = df_1["StartPoint"].str.replace(names[2], "경기")
df_1["StartPoint"] = df_1["StartPoint"].str.replace(names[3], "경상")
df_1["StartPoint"] = df_1["StartPoint"].str.replace(names[4], "전라")

In [10]:
df_1.head(2)

,date,StartPoint,Gyeonggi,Chungcheong,Jeolla,Gyeongsang,Gangwon
0,20140101,충청,83318.0,104617.0,4147.0,4126.0,6021.0
1,20140101,강원,57398.0,5783.0,427.0,991.0,102850.0


In [11]:
df = pd.concat([df_1, df_2])
df.head()

,date,StartPoint,Gyeonggi,Chungcheong,Jeolla,Gyeongsang,Gangwon
0,20140101,충청,83318.0,104617.0,4147.0,4126.0,6021.0
1,20140101,강원,57398.0,5783.0,427.0,991.0,102850.0
2,20140101,경기,373199.0,82263.0,3114.0,5841.0,51386.0
3,20140101,경상,6926.0,5368.0,15514.0,334217.0,986.0
4,20140101,전라,3298.0,4759.0,130474.0,14832.0,478.0


In [13]:
df.apply(lambda x: sum(x.isnull()))

date           0
StartPoint     0
Gyeonggi       0
Chungcheong    0
Jeolla         3
Gyeongsang     0
Gangwon        0
dtype: int64

#### [문제 1] 2014년 고속도로 통행 시작시점이 강원인 모든 차량 통행량(강원을 포함한 모든 도착지점)의 q1,median,q3 를 구하시오.
(정수형, 소수점 1자리에서 반올림) (10점)

In [16]:
df_q1 = df

In [17]:
df_q1_melt = df_q1.melt(id_vars = ["date", "StartPoint"])
df_q1_melt.head(2)

,date,StartPoint,variable,value
0,20140101,충청,Gyeonggi,83318.0
1,20140101,강원,Gyeonggi,57398.0


In [20]:
df_q1_melt.loc[df_q1_melt["StartPoint"] == "강원", "value"].quantile([0.25, 0.5, 0.75]).round(1)

0.25      988.0
0.50     5455.0
0.75    54882.0
Name: value, dtype: float64

#### [문제 2] 2014년 경기를 출발하여 경상에 도착한 교통량과 경상을 출발하여 경기에 도착한 교통량을 비교한다. 두 교통량의 평균이 통계적으로 유의한 차이가 있는지 유의수준 5% 수준에서 검정하시오. 
단, 일별로 짝을 짓지 않는 검정법을 사용한다. (10점)
- p_value는 소수 4째자리까지 기술한다(소수 4자리까지, 소수점 5자리에서 반올림)
- 귀무 가설(두 교통량의 평균은 차이가 없다)의 기각 여부를 YES 또는 NO 로 답한다. (대문자)

In [22]:
df_q2 = df
df_q2.head(2)

,date,StartPoint,Gyeonggi,Chungcheong,Jeolla,Gyeongsang,Gangwon
0,20140101,충청,83318.0,104617.0,4147.0,4126.0,6021.0
1,20140101,강원,57398.0,5783.0,427.0,991.0,102850.0


In [23]:
df_q2.loc[df_q2["StartPoint"] == "경기", "Gyeongsang"]

2     5841.0
7     6662.0
12    8137.0
Name: Gyeongsang, dtype: float64

In [26]:
stat, p_value = ttest_ind(df_q2.loc[df_q2["StartPoint"] == "경기", "Gyeongsang"],
                          df_q2.loc[df_q2["StartPoint"] == "경상", "Gyeonggi"])

In [27]:
round(p_value, 4)

0.0013

#### [문제 3] 2014년 충청을 출발하여 강원으로 도착한 교통량에 대해서, 요일별 평균 교통량을 월 별로 정리한다. 요일별로 표준화(Min-Max Normalization)를 진행한 후, k가 3인 K-means 군집화를 수행한다.(iteration 수는 100회로 한다.) 10월과 같은 군집에 포함된 월 수(10월을 포함)와 해당 군집 월의 월요일 교통량의 평균 값(표준화된 값의 평균을 사용)을 구하시오. (10점)
 - 월 수는 정수형, 평균값은 소수 2자리까지, 3자리에서 반올림
 - Brightics의 경우요일변수 생성은 SQL 에서 date_format(timestamp,'u') 함수를 사용할 수 있음. <br> 
    ex) select date_format(‘2018-07-13’, ‘u’) from #{DF(0)}
 - R의 경우 weekdays 함수를 사용할 수 있음.  ex) weekdays(as.Date(‘2018-07-13’)))

In [34]:
df_q3 = df
df_q3.head(2)

,date,StartPoint,Gyeonggi,Chungcheong,Jeolla,Gyeongsang,Gangwon
0,20140101,충청,83318.0,104617.0,4147.0,4126.0,6021.0
1,20140101,강원,57398.0,5783.0,427.0,991.0,102850.0


In [35]:
df_q3 = df_q3.loc[df_q3["StartPoint"] == "충청", ["date", "Gangwon"]]
df_q3.head(2)

,date,Gangwon
0,20140101,6021.0
5,20140102,5270.0


In [43]:
print(pd.to_datetime(df_q3["date"], format = "%Y%m%d")[:3])
print(pd.to_datetime(df_q3["date"], format = "%Y%m%d")[:3].dt.weekday)

0    2014-01-01
5    2014-01-02
10   2014-01-03
Name: date, dtype: datetime64[ns]
0     2
5     3
10    4
Name: date, dtype: int64


In [44]:
df_q3["date"] = pd.to_datetime(df_q3["date"], format = "%Y%m%d")
df_q3["month"] = df_q3["date"].dt.month
df_q3["wday"] = df_q3["date"].dt.weekday

In [45]:
df_q3.head(2)

,date,Gangwon,month,wday
0,2014-01-01,6021.0,1,2
5,2014-01-02,5270.0,1,3


In [85]:
df_q3_pivot = df_q3.pivot_table(index = "month", columns = "wday", values = "Gangwon", aggfunc = "mean")
df_q3_pivot.head(2)

wday,0,1,2,3,4,5,6
month,,,,,,,
1,5094.5,5066.75,5635.2,5397.0,6253.40,6749.0,4552.50
2,5131.0,5064.75,5227.0,5284.5,6273.25,6245.5,4382.75


In [87]:
df_q3_pivot = df_q3_pivot.apply(lambda x: (x - min(x))/(max(x) - min(x)))
df_q3_pivot

wday,0,1,2,3,4,5,6
month,,,,,,,
1,0.000000,0.001116,0.243047,0.088929,0.000000,0.221018,0.099979
2,0.018383,0.000000,0.028628,0.031596,0.007848,0.000000,0.000000
3,0.091614,0.042405,0.000000,0.000000,0.005080,0.247092,0.201343
4,0.219592,0.143088,0.163835,0.090712,0.101546,0.402199,0.241187
5,0.425837,0.236295,0.256861,0.292674,0.272713,0.608885,0.643019
6,0.320675,0.330869,0.290348,0.492292,0.548292,0.556933,0.331390
7,0.585621,0.458781,0.456100,0.501414,0.398355,0.606646,0.592220
8,1.000000,1.000000,1.000000,1.000000,1.000000,0.924674,1.000000
9,0.671619,0.330004,0.120420,0.442604,0.289436,0.602476,0.833260


In [88]:
model = KMeans(n_clusters = 3, max_iter = 100).fit(df_q3_pivot)
model

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
       n_clusters=3, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [89]:
model.labels_

array([0, 0, 0, 0, 2, 2, 2, 1, 2, 1, 1, 2])

In [90]:
df_q3_pivot["cluster"] = model.labels_
df_q3_pivot

wday,0,1,2,3,4,5,6,cluster
month,,,,,,,,
1,0.000000,0.001116,0.243047,0.088929,0.000000,0.221018,0.099979,0
2,0.018383,0.000000,0.028628,0.031596,0.007848,0.000000,0.000000,0
3,0.091614,0.042405,0.000000,0.000000,0.005080,0.247092,0.201343,0
4,0.219592,0.143088,0.163835,0.090712,0.101546,0.402199,0.241187,0
5,0.425837,0.236295,0.256861,0.292674,0.272713,0.608885,0.643019,2
6,0.320675,0.330869,0.290348,0.492292,0.548292,0.556933,0.331390,2
7,0.585621,0.458781,0.456100,0.501414,0.398355,0.606646,0.592220,2
8,1.000000,1.000000,1.000000,1.000000,1.000000,0.924674,1.000000,1
9,0.671619,0.330004,0.120420,0.442604,0.289436,0.602476,0.833260,2


In [95]:
df_q3_pivot["cluster"].value_counts()

2    5
0    4
1    3
Name: cluster, dtype: int64

In [93]:
df_q3_pivot.loc[df_q3_pivot["cluster"] == 1, 0].mean()

0.7506085788634266

In [94]:
round(df_q3_pivot.loc[df_q3_pivot["cluster"] == 1, 0].mean(), 2)

0.75

#### [문제 4] 경기를 출발하여 전라로 도착한 교통량 정보 중 14년 27주차, 28주차, 그리고 29주차 일요일(20140706, 20140713, 20140720) 정보가 삭제되었다. 삭제된 교통량을 선형 회귀 분석을 이용하여 예측한 값으로 채워 넣으려고 한다.
 - 종속 변수(Y)는 경기를 출발하여 전라에 도착한 일 교통량
 - 독립 변수(X)는 <br> 
 ① 당일 경기를 출발하여 충청에 도착한 교통량 <br> 
 ② 당일 경기를 출발하여 경상에 도착한 교통량 <br> 
 ③ 당일 경기를 출발하여 강원에 도착한 교통량 <br> 
 ④ 전일(일요일 기준 토요일) 경기를 출발하여 전라에 도착한 교통량으로 한다.  <br> 
 <br> 
 - 회귀 분석의 학습 데이터는 1월에서 6월까지의 일요일을 기준으로 사용
 - 예측은 삭제된 일요일 3일인 20140706, 20140713, 20140720에 대하여 수행
 - 예측된 3일의 예측 값을 차례대로 구하시오. (20점)

In [96]:
df_q4 = df
df_q4.head(2)

,date,StartPoint,Gyeonggi,Chungcheong,Jeolla,Gyeongsang,Gangwon
0,20140101,충청,83318.0,104617.0,4147.0,4126.0,6021.0
1,20140101,강원,57398.0,5783.0,427.0,991.0,102850.0


In [98]:
df_q4 = df_q4[df_q4["StartPoint"] == "경기"]
df_q4.head(2)

,date,StartPoint,Gyeonggi,Chungcheong,Jeolla,Gyeongsang,Gangwon
2,20140101,경기,373199.0,82263.0,3114.0,5841.0,51386.0
7,20140102,경기,517935.0,89221.0,3046.0,6662.0,43191.0


In [99]:
df_q4["date"] = pd.to_datetime(df_q4["date"], format = "%Y%m%d")
df_q4["month"] = df_q4["date"].dt.month
df_q4["wday"] = df_q4["date"].dt.weekday
df_q4.head(2)

,date,StartPoint,Gyeonggi,Chungcheong,Jeolla,Gyeongsang,Gangwon,month,wday
2,2014-01-01,경기,373199.0,82263.0,3114.0,5841.0,51386.0,1,2
7,2014-01-02,경기,517935.0,89221.0,3046.0,6662.0,43191.0,1,3


In [142]:
df_q4["Jeolla_yes"] = df_q4["Jeolla"].shift(1)
df_q4.head(2)

,date,StartPoint,Gyeonggi,Chungcheong,Jeolla,Gyeongsang,Gangwon,month,wday,Jeolla_yes
2,2014-01-01,경기,373199.0,82263.0,3114.0,5841.0,51386.0,1,2,NaN
7,2014-01-02,경기,517935.0,89221.0,3046.0,6662.0,43191.0,1,3,3114.0


In [143]:
train = df_q4.loc[(df_q4["month"] <= 6) & (df_q4["wday"] == 6), ]
train = train.iloc[:, [2, 3, 4, 5, 6, 9]]
train.head()

,Gyeonggi,Chungcheong,Jeolla,Gyeongsang,Gangwon,Jeolla_yes
22,359233.0,74418.0,3058.0,5703.0,40280.0,4413.0
57,369717.0,76985.0,2819.0,5283.0,42210.0,4536.0
92,379265.0,81428.0,2818.0,5560.0,43549.0,4354.0
127,361135.0,71975.0,2437.0,4358.0,37041.0,3818.0
162,287950.0,59596.0,3316.0,7270.0,30180.0,4046.0


In [144]:
test = df_q4.loc[df_q4["Jeolla"].isnull(), :]
test = test.iloc[:, [2, 3, 5, 6, 9]]
test

,Gyeonggi,Chungcheong,Gyeongsang,Gangwon,Jeolla_yes
932,420229.0,84761.0,5460.0,44771.0,4837.0
967,432308.0,85702.0,5492.0,47001.0,5082.0
1002,424885.0,82962.0,5356.0,47133.0,4434.0


In [145]:
model = ols("Jeolla ~ " + "+".join(train.columns[[0, 1, 3, 4, 5]]), data = train)
model_fit = model.fit()
model_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Jeolla   R-squared:                       0.989
Model:                            OLS   Adj. R-squared:                  0.986
Method:                 Least Squares   F-statistic:                     348.8
Date:                Fri, 23 Aug 2019   Prob (F-statistic):           9.79e-19
Time:                        15:15:07   Log-Likelihood:                -175.70
No. Observations:                  26   AIC:                             363.4
Df Residuals:                      20   BIC:                             371.0
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept    -335.0865    718.676     -0.466      0.646   -1834.219    1164.046
Gyeonggi       -0.0003      0.004     -0.076      0.940      -0.009       0.008
Chungcheong    -0.0438      0.022     -2.006      0.059      -0.089       0.002
Gyeongsang      0.4801      0.067      7.216      0.000       0.341       0.619
Gangwon         0.0673      0.025      2.723      0.013       0.016       0.119
Jeolla_yes      0.3097      0.049      6.308      0.000       0.207       0.412
==============================================================================
Omnibus:                        2.158   Durbin-Watson:                   2.679
Prob(Omnibus):                  0.340   Jarque-Bera (JB):                0.982
Skew:                           0.419   Prob(JB):                        0.612
Kurtosis:                       3.451   Cond. No.                     6.38e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.38e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [146]:
model_fit.predict(test)

932     2958.896604
967     3155.450842
1002    3020.628481
dtype: float64